In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import gc
import os
import sklearn
import sys

from joblib import Parallel, delayed
from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [ ]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../src')
sys.path.append(src_dir)

In [ ]:
%aimport data.delicious_t140

In [ ]:
from data.delicious_t140 import load_taginfo_into_dataframe

In [ ]:
ROOT = "/media/felipe/SAMSUNG/delicious/delicioust140/"

In [ ]:
TAGINFO=ROOT+"taginfo.xml"

In [ ]:
df = load_taginfo_into_dataframe(TAGINFO)

In [ ]:
docs_df=df

In [ ]:
docs_df.describe(include=['number'])

In [ ]:
docs_df.describe(include=['category','object'])

In [ ]:
docs_df.dtypes

In [ ]:
docs_df.sample(10)

In [ ]:
docs_df[docs_df['num_users']<=2].sort_values('num_users').head(20)

In [ ]:
# number of documents by type

fig = plt.gcf()

fig.set_size_inches(15,10)

grouped = docs_df.groupby('filetype')['hash'].size()

print(grouped)

grouped.plot(kind='bar')

In [ ]:
# mean num_tags by type
docs_df.groupby('filetype')['num_tags'].mean().plot(kind='bar')

In [ ]:
# mean num_users by type

grouped = docs_df.groupby('filetype')['num_users'].mean()

print(grouped)

grouped.plot(kind='bar')

In [ ]:
max(set(docs_df['num_users'].values))

In [ ]:
plt.clf()
docs_df.groupby('num_users').count()['hash'].sort_values(ascending=False).plot(kind='bar')

fig = plt.gcf()
fig.set_size_inches(18,5)

plt.gca().set_title('number of users per resource')
plt.gca().set_xlim(-1,100)

In [ ]:
len(docs_df[docs_df['num_tags'] == 25])

In [ ]:
docs_df.groupby('num_tags').count()['hash'].sort_values(ascending=False).plot(kind='bar')
plt.gca().set_title('number of tags per resource')
plt.gca().set_xlim(-1,30)

In [ ]:
# total tags

tagsets = [tagset.split(',') for tagset in docs_df["tags"].values]

flat_list = []

for tagset in tagsets:
    for tag in tagset:
        flat_list.append(tag)
        
flat_set = set(flat_list)

In [ ]:
len(flat_set)

In [ ]:
tags = [tag for tag in sorted(flat_set) if "-" in tag]
tags

In [ ]:
len(tags)